In [1]:
import collections

In [2]:
import csv

In [3]:
import itertools

In [4]:
import numpy as np

In [5]:
import os

In [6]:
import scipy.special

In [7]:
import scipy.stats as stats

In [8]:
from statsmodels.sandbox.stats.multicomp import fdrcorrection0

In [9]:
from sklearn import metrics

---

In [10]:
os.chdir('/work/jyoung')

In [11]:
import pyuserfcn

In [12]:
os.chdir('/work/jyoung/genetic_interact/src')

In [13]:
import func_net_pred

---

**2015 September 9**

Start with *Saccharomyces cerevisiae*. For predictive seed sets (0.8 &le; AUC < 1.0), examine all possible pairs and count the number of genetic interactions between them. Execute for each type of genetic interaction. Seed sets are from BIOGRID v3.4.127 with all interactions before 2007 removed. 

**2015 September 10**

It is not feasible to only examine all of the counts. There are 240 seed gene sets with 0.8 &le; AUC < 1.0, which yields 28680 total pairs - far too many to examine one-by-one. Try using a binomial distribution to calculate the significance of interaction between seed sets. Suppose the number of genes in the 1<sup>st</sup> seed set is *n*<sub>1</sub>, the number of genes in the 2<sup>nd</sup> seed set is n<sub>2</sub>, and the number of interactions between the sets is *k*. The question is what to use for the probability *p* of a "success." Should it be 1 divided by the total number of all interacting pairs? The overall total number of pairs that were tested is not known, so it is not possible to calculate the number of interacting pairs divided by the number tested. Could the number of genes in the interaction set choose 2 be used instead? 

In [14]:
experimentSys = 'Dosage Growth Defect'

In [15]:
node2edgewt = func_net_pred.process_func_net()
gene2idx = func_net_pred.assign_gene_indices(node2edgewt)

In [16]:
matrixPath = '/work/jyoung/genetic_interact/data/YeastNet2_adj_matrix.npy'
adjMat = np.load(matrixPath)

In [13]:
len(gene2idx.keys())  # number of genes in functional net

5483

In [17]:
seedSets = func_net_pred.read_biogrid(experimentSys)

Number of genes in interactions: 1192


In [18]:
seedAUC, seed2interactors = func_net_pred.seed_set_predictability(gene2idx, adjMat, seedSets)

Create a list containing predictive seed genes.

In [19]:
lowerAUC = 0.8
upperAUC = 1.0

In [20]:
predictiveSeeds = list()
for p in seedAUC:  # p=(AUC, gene)
    if p[0] >= lowerAUC and p[0] < upperAUC:
        predictiveSeeds.append(p[1])

In [23]:
len(predictiveSeeds)

240

Now assemble all the genetic interactions into a single set. 

In [21]:
interactPairs = set()
for seed in predictiveSeeds:
    for interactor in seed2interactors[seed]:
        interactPairs.update([(seed, interactor), (interactor, seed)])

In [19]:
len(interactPairs)  # number of interacting pairs

1624

Compute statistics for each pair of interactor sets.

In [22]:
p = len(interactPairs)/scipy.special.binom(1192, 2)
results = list()
for seedPair in itertools.combinations(predictiveSeeds, 2):  # seedPair=(seed1, seed2)
    interactionCount = 0
    num1stSet = len(seed2interactors[seedPair[0]])
    num2ndSet = len(seed2interactors[seedPair[1]])
    for genePair in itertools.product(seed2interactors[seedPair[0]], seed2interactors[seedPair[1]]):
        if genePair in interactPairs:
            interactionCount += 1
    n = num1stSet * num2ndSet
    pval = stats.binom.pmf(interactionCount, n, p) + stats.binom.sf(interactionCount, n, p)
    results.append((num1stSet, num2ndSet, interactionCount, pval))

In [21]:
len(results)

28680

Is there any statistical significance at some FDR level? Try using Benjamini-Hochberg to control FDR at 5%. 

In [23]:
pvals = [x[3] for x in results]

In [24]:
rejected, pvalsCor = fdrcorrection0(pvals)

In [28]:
np.sum(rejected)

2167

Create directory to store text files in CSV format containing results.

    cd /work/jyoung/genetic_interact/results
    mkdir YeastInteractClust

In [33]:
os.chdir('/work/jyoung/genetic_interact/results/YeastInteractClust')

In [34]:
with open(''.join(experimentSys.split()) + 'Stats.txt', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['# genes in 1st set', '# genes in 2nd set', '# interactions btw sets', 'p-value'])
    for t in results:
        csvwriter.writerow(t)

Put the code above into a script so that results can be easily output for a given species and genetic interaction type. 